## MF731 Homework2 Part 2
### Edited by Xuyang Liu

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm

### 2. Time aggregated risk measures for a constant weight portfolio of equities.

In [8]:
#(b)
alpha = 0.99
gamma = 30
lamda = 0.94
theta = 0.97
N = 50000

data = pd.read_csv('Prices.csv',index_col=0)
logret = np.log(data/data.shift(1)).dropna()

In [13]:
# for initial mu
mulst = np.zeros([len(logret),4])
for i in range(1,len(logret)):
    mulst[i,:] = lamda * mulst[i-1,:] + (1-lamda) * logret.iloc[i,:]
mu = mulst[-1]

In [16]:
mu

array([-0.00044666,  0.00320015,  0.00423529,  0.00353126])

In [116]:
# for the newest covariance matrix
sigma = np.mat([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
for i in range(1,len(logret)):
    sigma = theta * sigma + (1-theta) * ( np.mat(logret.iloc[i,:].values)-np.mat(mulst[i-1,:]) ).T * ( np.mat(logret.iloc[i,:].values)-np.mat(mulst[i-1,:]) )

In [117]:
sigma

matrix([[1.92425384e-03, 3.75255309e-04, 5.32993246e-04, 3.30030591e-05],
        [3.75255309e-04, 2.61346684e-04, 1.87880003e-04, 4.64075474e-05],
        [5.32993246e-04, 1.87880003e-04, 3.98542802e-04, 2.21414843e-05],
        [3.30030591e-05, 4.64075474e-05, 2.21414843e-05, 2.78812901e-04]])

In [19]:
# For constant weight vector
w1 = 97.39 / (97.39 + 158.20 + 179.01 + 417.97)
w2 = 158.20 / (97.39 + 158.20 + 179.01 + 417.97)
w3 = 179.01 / (97.39 + 158.20 + 179.01 + 417.97)
w4 = 417.97 / (97.39 + 158.20 + 179.01 + 417.97)

w = np.array([w1,w2,w3,w4])

In [93]:
def estimate(X,mu,sigma):
    new_mu = lamda * mu + (1-lamda) * X
    new_sigma = theta * sigma + (1-theta) * (np.mat(X)-np.mat(mu)).T * (np.mat(X)-np.mat(mu))
    return new_mu, new_sigma
    

In [100]:
def simulation(N,K,init_mu,init_sigma):
    losslst = []
    for i in range(N):
        #lossday = []
        mu , sigma = init_mu , init_sigma
        Xt = 0
        for j in range(K):
            X = np.random.multivariate_normal(mu,sigma)
            Xt += X
            mu,sigma = estimate(X,mu,sigma)
            
        loss = - np.dot((np.exp(Xt)-1),w)
        losslst.append(loss * 1000000)
            
        
        
    VaR = pd.Series(losslst).quantile(alpha)
    lossar = np.array(losslst)
    ES = 1/(1-alpha)/N * sum(lossar[lossar>=VaR])
    
    return VaR, ES

In [124]:
onedayVaR,onedayES = simulation(N,1,mu,sigma)
tendayVaR,tendayES = simulation(N,10,mu,sigma)
multVaR, multES = 10**(0.5) * onedayVaR, 10**(0.5) *onedayES

In [125]:
print("The one day VaR and ES are about: ",onedayVaR," ",onedayES)
print("The ten day VaR and ES are about: ",tendayVaR," ",tendayES)
print("The Square root of time of the VaR and ES are about: ",multVaR," ",multES)

The one day VaR and ES are about:  28456.340712679   32581.691413407152
The ten day VaR and ES are about:  92992.93940697976   110871.9060868864
The Square root of time of the VaR and ES are about:  89986.85052584475   103032.35488711736
